# Regression Analysis with Boston House Dataset in Keras

In this notebook we will cover:

* How to create simple regressor NN model and work with continuous numerical data
* Using K-Fold cross validation in order to get more general results (Sklearn library used for that)
* Mean Squared Error (MSE) and Mean Absolute Error (MAE)

### Required Modules

In [ ]:
from keras.datasets import boston_housing
import numpy as np
from keras import models, layers
import matplotlib.pyplot as plt
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold

### Loading and Exploring Dataset

In [ ]:
(train_data, train_labels), (test_data, test_labels) = boston_housing.load_data()

57344/57026 [==============================] - 0s 0us/step


* In this case, we have limited dataset as 404 train, 102 test samples

In [ ]:
train_data.shape

(404, 13)

In [ ]:
test_data.shape

(102, 13)

* Labels consist of prices of the houses at Boston in mid-1970s.
* They were written in short form as 50 which represents 50000$

In [ ]:
train_labels[:10]

array([15.2, 42.3, 50. , 21.1, 17.7, 18.5, 11.3, 15.6, 15.6, 14.4])

### Normalizing Data

* StandardScaler in Scikit-learn library normalizes data in feature based insted sample base.

In [ ]:
std = StandardScaler()

train_data = std.fit_transform(train_data)
test_data = std.transform(test_data)

In [ ]:
train_data[0]

array([-0.27224633, -0.48361547, -0.43576161, -0.25683275, -0.1652266 ,
       -0.1764426 ,  0.81306188,  0.1166983 , -0.62624905, -0.59517003,
        1.14850044,  0.44807713,  0.8252202 ])

* Before start fitting model lets seperate 100 sample as validation data from training

In [ ]:
X_train = train_data[100:]
y_train = train_labels[100:]
X_val = train_data[:100]
y_val = train_labels[:100]

### Building and Fitting Model

Since we will be using same model many times later, we have built it inside function:

* Our model consist of 4 hidden layer with 128, 64, 16 and 4 units. Rectified Linear Unit `ReLu` activation function is used.
* `rmsprop` used as optimizer. However, you might try `adam` or `sgd` as well. Besides, `tanh` activation funtion may count as another option.

In [ ]:
def build_model():
  model = models.Sequential()
  model.add(layers.Dense(128, activation='relu', input_shape=(train_data.shape[1],)))
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(16, activation='relu'))
  model.add(layers.Dense(4, activation='relu'))
  model.add(layers.Dense(1)) # for regression there is no activation and layer has single unit

  model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
  return model

In [ ]:
model = build_model()
history = model.fit(X_train, y_train, epochs=20, batch_size=4, validation_data=(X_val, y_val))

Epoch 1/20
76/76 [==============================] - 0s 6ms/step - loss: 200.7920 - mae: 10.9542 - val_loss: 34.8257 - val_mae: 3.9821
Epoch 2/20
76/76 [==============================] - 0s 2ms/step - loss: 31.2657 - mae: 3.9388 - val_loss: 22.4652 - val_mae: 3.3547
Epoch 3/20
76/76 [==============================] - 0s 3ms/step - loss: 21.0626 - mae: 3.1302 - val_loss: 16.4121 - val_mae: 2.8201
Epoch 4/20
76/76 [==============================] - 0s 3ms/step - loss: 18.1595 - mae: 2.8867 - val_loss: 15.8029 - val_mae: 2.7889
Epoch 5/20
76/76 [==============================] - 0s 3ms/step - loss: 15.7975 - mae: 2.7387 - val_loss: 11.3968 - val_mae: 2.2820
Epoch 6/20
76/76 [==============================] - 0s 3ms/step - loss: 13.4953 - mae: 2.5800 - val_loss: 11.2366 - val_mae: 2.2701
Epoch 7/20
76/76 [==============================] - 0s 4ms/step - loss: 13.4130 - mae: 2.5274 - val_loss: 9.8178 - val_mae: 2.1030
Epoch 8/20
76/76 [==============================] - 0s 3ms/step - loss: 12.

* At the end of 20 epoch we got roughly 1.97 validation mean absolute error which equivalent to 1970\$ error in validation set.
* However, in test case we have 2720\$ error. That makes 750\$ gap between validation and test scores. That means model tents to overfit validation set.

In [ ]:
model.evaluate(test_data, test_labels)

4/4 [==============================] - 0s 2ms/step - loss: 19.0536 - mae: 2.7212


[19.053613662719727, 2.7211787700653076]

### Validating Model Using KFold

* Previously, we used 100 sample as validation data and we observed that data tents to overfit validation data. During KFold we are splitting data in 10 portion that makes 364 training and 40 test sample for each fold.
* Now, lets use 10-Fold cross validation to obtain average test error

In [ ]:
kf = KFold(n_splits=10, shuffle=True, random_state=1)
val_scores = []
test_scores = []

for train_index, test_index in kf.split(train_data, train_labels):
  X_train, X_val = train_data[train_index], train_data[test_index]
  y_train, y_val = train_labels[train_index], train_labels[test_index]

  model = build_model()
  history = model.fit(X_train, y_train, epochs=20, batch_size=4, verbose=0, validation_data=(X_val, y_val))
  val_scores.append(history.history['val_mae'][-1])
  test_scores.append(model.evaluate(test_data, test_labels)[1])

4/4 [==============================] - 0s 3ms/step - loss: 18.6407 - mae: 2.7264


As a new results, we have 500\$ gap between validation and test data since we decrease the amount of validation data. So, that is unlikely overfit it

In [ ]:
np.mean(np.abs(np.ravel(test_scores)-np.ravel(val_scores)))

0.5034162402153015

* We have roughly 2850$ test error in average. That means we hurt the test performance a bit.

In [ ]:
np.mean(test_scores)

2.8533445835113525

Now, since we have limited data, let's repeate the same process without using validation data. But in this case we will concatenate train and test samples in order to evaluate our model with different portion of whole data-set using 5-fold cross validation.

The main reason we chose 5-fold cross validation is 1 portion of it come across almost 100 sample as test data, which is same amount with previous case.

In [ ]:
X = np.concatenate((train_data, test_data), axis=0)
y = np.concatenate((train_labels, test_labels), axis=0)

In [ ]:
X.shape

(506, 13)

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=1)
scores = []

for train_index, test_index in kf.split(X, y):
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = y[train_index], y[test_index]

  model = build_model()
  model.fit(X_train, y_train, epochs=20, batch_size=4, verbose=0)
  scores.append(model.evaluate(X_test, y_test)[1])

4/4 [==============================] - 0s 2ms/step - loss: 10.5325 - mae: 2.4890


In [ ]:
scores

[2.794982671737671,
 2.3391175270080566,
 2.690911054611206,
 2.5324416160583496,
 2.4890496730804443]

Now, we have obtained 2.57 MAE score and 2570$ overall deviation which is better than our previous cases.

In [ ]:
np.mean(scores)

2.5693005084991456

### One More Thing!

In Kaggle compatition of [Boston Housing](https://www.kaggle.com/c/boston-housing/overview) they were looking at root mean squared error `RMSE`, let's change our evaluation metric with rmse and re-train our model.

In [ ]:
from keras import backend as K
import tensorflow as tf

@tf.function
def RMSE(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 

def build_model():
  model = models.Sequential()
  model.add(layers.Dense(128, activation='relu', input_shape=(train_data.shape[1],)))
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(16, activation='relu'))
  model.add(layers.Dense(4, activation='relu'))
  model.add(layers.Dense(1))

  model.compile(optimizer='rmsprop', loss='mse', metrics=RMSE)
  return model

Since tensorflow creates randomly initialized weights in each time, let's run our model 10 times and takes the mean of them.

In [ ]:
rmse_scores = []
for step in range(10):
  model = build_model()
  model.fit(train_data, train_labels, epochs=20, batch_size=4, verbose=0)
  rmse_scores.append(model.evaluate(test_data, test_labels)[1])

4/4 [==============================] - 0s 2ms/step - loss: 22.1246 - RMSE: 2.8887


In [ ]:
print('RMSE Score:',np.mean(rmse_scores))

RMSE Score: 2.8063689708709716


We got 2.8 rmse score in average which places us within top 5 in the leaderbord even without making any feature engineering.